In [ ]:
from matplotlib.pyplot import *
from numpy import *
import numpy.random as npr
rcParams['text.usetex'] = True

In [ ]:
from freecad.optics_design_workbench import distributions
from freecad.optics_design_workbench.distributions import *

# Test random sample generators for given distribution function (true random)

### test standard gaussian

In [ ]:
v = distributions.VectorRandomVariable(
          probabilityDensity='exp(-(theta/sigma)**2)',
          variables=['theta', 'phi'],
          variableDomains=[(0, inf), (0, 2*pi)])

In [ ]:
v.compile(sigma=.1)

In [ ]:
v.showExpressions()

In [ ]:
v.mode()

In [ ]:
%%time
# draw one true random sample
v.draw()

In [ ]:
%%time
# draw true random samples
v.draw(N=10)

In [ ]:
%%time
# draw true random samples
thetas, phis = v.draw(N=500)

In [ ]:
# draw random samples that match mean expected histogram as well as possible
#v.drawPseudo(N=10, rerollCount=10)

In [ ]:
# make mesh with point spacings approximating the inverse probability density as well as possible 
#v.mesh1D(meshVar='theta', otherVars=dict(phi=(0, pi/3, 2*pi/3)))

In [ ]:
# make mesh with triangle areas approximating the inverse probability density as well as possible
#v.mesh2D(meshVars=('theta', 'phi'), othervars={})

### test top hat

In [ ]:
gen = distributions.VectorRandomVariable(
          probabilityDensity='1',
          variables=['theta', 'phi'],
          numericalResolutions=[100, 10],
          variableDomains=[(0, 0.1), (0, 2*pi)])

In [ ]:
gen.compile()
gen.showExpressions()

In [ ]:
gen.draw()

### test negative probability density

In [ ]:
gen = distributions.VectorRandomVariable(
          probabilityDensity='cos(phi)',
          variables=['theta', 'phi'],
          variableDomains=[(0, pi), (0, 2*pi)])

In [ ]:
gen.compile()

In [ ]:
gen.showExpressions()

In [ ]:
gen.draw()

### test offset cosine

In [ ]:
x = distributions.ScalarRandomVariable('cos(x)+2', 'x', (0,pi))

In [ ]:
x.compile()

### test elliptical gaussian

In [ ]:
gen = distributions.VectorRandomVariable(
          probabilityDensity='exp(-(theta/(sigma*(1+0.5*cos(phi + pi/3))))**2)',
          variables=['theta', 'phi'],
          variableDomains=[(0, pi), (0, 2*pi)])

In [ ]:
gen.compile()

In [ ]:
import sympy as sy
import signal

In [ ]:
def handler(sig, n):
  print('jo')
  raise ValueError('bre')

In [ ]:
signal.signal(signal.SIGALRM, handler)

In [ ]:
#expr = sy.sympify('(sin(10*theta)/theta)^2')
#expr = sy.sympify('exp(-theta**2)') + sy.sympify('exp(-.1*theta**2)')
#expr = sy.sympify('Piecewise((1, abs(theta)<1), (0,True))')
#expr = sy.sympify('1/(1+abs(theta)**2)')
expr = sy.sympify('1/theta')

In [ ]:
l1, l2 = 1e-9, 2
l1, l2 = -sy.oo, sy.oo

In [ ]:
expr

In [ ]:
var = list(expr.free_symbols)[0]

In [ ]:
x, y = sy.symbols('x y', real=True, positive=True)

In [ ]:
sy.Integral(expr, (var,l1,x)).doit()

In [ ]:
sy.Integral(expr, (var,l1,l2)).doit()

In [ ]:
integr = sy.Integral(expr, (var,l1,x)).doit() / sy.Integral(expr, (var,l1,l2)).doit()

In [ ]:
integr

In [ ]:
signal.alarm(3)
Y = sy.solve(sy.Eq(integr, y), x)[0]

In [ ]:
Y

In [ ]:
Yl = sy.lambdify(y, Y)

In [ ]:
Yl(3)

In [ ]:
i = linspace(0, 1, 30000)
I = Yl(i)

In [ ]:
plot(i, I)

# Test random sample generators for given distribution function (pseudo random)

# Test 1D point generators for given densities (ray fans)

In [ ]:
Xrand = npr.normal(5, 1, size=150)
hX, hDensity = calcHistDensity(Xrand)
dX, dDensity = calcDiffDensity(Xrand)
plot(hX, hDensity, '.')
plot(dX, dDensity, '.')
xlabel('x')
ylabel('density')

### Gaussian

In [ ]:
X = linspace(-1, 2, 500)
Y = exp(-5*X**2)
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=20)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'beliebige Verteilungsfunktion\\als X,Y arrays angegeben')
xlabel('x')
ylabel('normierte Dichte')
plot(Xgen, [0]*len(Xgen), 'o', label=r'platzierte Punkte')
plot(dX, dDens/max(dDens), 'x', label=r'aus den Punktabst\"anden\\bestimmte Dichte')
legend()
#io.savefig('~/Desktop/gaussian-platzierung.jpg')

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-2

### Tophat

In [ ]:
X = linspace(-1, 2, 500)
Y = arctan( 1e5 * ( exp(-5*X**2) - .5 ) )/pi + .5
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=10)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-5

### tests distorted gaussian

In [ ]:
X = linspace(-1, 3, 500)
Y = arctan( 20*exp(-5*X**2) ) * (1+X)
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=25)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-2

### Test distribution with multiple maxima

In [ ]:
X = linspace(-1, 5, 500)
Y = exp(-5*X**2) + 0.7*exp(-5*(X-2)**2) 
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=30)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 3e-2

### same with more points

In [ ]:
X = linspace(-1, 5, 500)
Y = exp(-5*X**2) + 0.7*exp(-5*(X-2)**2)
Xgen = generatePointsWithGivenDensity1D(density=(X,Y), N=90)
dX, dDens = calcDiffDensity(Xgen)

In [ ]:
plot(X, Y/max(Y), label=r'given density')
xlabel('x')
ylabel('normalized density')
plot(Xgen, [0]*len(Xgen), 'o', label=r'generated points')
plot(dX, dDens/max(dDens), 'x', label=r'\parbox{4cm}{density reconstructed from point-distance}')
legend()

In [ ]:
rmsErr = sqrt(mean([abs((Y/max(Y))[argmin(abs(X-x))]-y)**2 for x,y in zip(dX, dDens/max(dDens))]))
print(rmsErr)
assert rmsErr < 1e-2